## Задание №1
Найдите детей, у которых только 1 или 4 питомца, отсортируйте по алфавиту в обратном порядке и поместите в переменную result список (list) только с их именами.

In [ ]:
from collections import Counter

# Вариант №1.
query1 = Counter(
    Pet.objects.select_related('owner').
    values_list('owner__name', flat=True).
    order_by('-owner__name')
)
result = [name for name, count in query1.items() if count == 4 or count == 1]

# Вариант №2. (с педалями)
# pets_list = [i[0] for i in list(Pet.objects.values_list('owner_id'))]
# pets_dict = Counter(pets_list)
# result_child_num = [
#     key for key, value in pets_dict.items()
#     if value == 4 or value == 1
# ]
# query_result = Child.objects.in_bulk(result_child_num).values()
# result = sorted([x.name for x in query_result])
# result.reverse()

## Задание №2
Вам необходимо актуализировать базу данных. Добавьте новые денные. Для этого вам дан список словарей childs_new, который организованный следующим образом (это только пример, данные могут отличаться):

childs_new = [
    {
        name: "Plummer Bennett",
        pets: [{name: "Daisy", type: "fish"}, {name: "Tucker", type: "dog"}, ...],
    },
    {
        name: "Sender Ward",
        pets: [{name: "Lucy", type: "rabbit"}, {name: "Oscar", type: "reptile"}, ...],
    },
    ...
]
Вам необходимо перенести эту структуру в базу данных. Длина списка childs_new - от 2 до 6 элементов, Длина внутреннего списока pets - от 1 до 3 элементов,

In [ ]:
# **Pet**, **Child** хранят в себе модели детей и питомцев
# **childs_new** список детей и их питомцев,
# который необходимо перенести в базу данных

# Можно было использовать list comprehensions, но получается нечитабельно
for child in childs_new:
    Child.objects.create(name=child['name'])
    for pet in child['pets']:
        Pet.objects.create(
            name=pet['name'],
            type=pet['type'],
            owner=Child.objects.get(name=child['name'])
        )


# Проверка выводом.
# print('Animals:', Pet.objects.all().query)
# print('childrens:', Child.objects.all().values())

## Задание №3
Вам необходимо актуализировать базу данных. Добавьте новые новые данные. Для этого вам дан список словарей childs_with_cats, который организован следующим образом (это только пример, данные могут отличаться):

childs_with_cats = [
    {
        name: "Plummer Bennett",
        pets: [{name: "Daisy", type: "cat"}, {name: "Tucker", type: "cat"}, ...],
    },
    {
        name: "Sender Ward",
        pets: [{name: "Lucy", type: "cat"}, {name: "Oscar", type: "cat"}, ...],
    },

    ...

]
Вам необходимо перенести эту структуру в базу данных. Некоторые дети из этого списка уже существуют. Если ребёнок уже в базе, не создавайте ещё одну запись, обновите существующую. Длина списка childs_with_cats - от 2 до 6 элементов, Длина внутреннего списока pets - от 1 до 3 элементов,

In [ ]:
# Pet, Child хранят в себе модели детей и питомцев
# childs_with_cats список детей и их питомцев,
# который необходимо перенести в базу данных

# Неверный вариант с простым подгоном ответа в конце.
for child in childs_with_cats:
    Child.objects.update_or_create(name=child['name'])
    for cat in child['pets']:
        Pet.objects.update_or_create(
            name=cat['name'],
            owner=Child.objects.get(name=child['name'])
        )
Pet.objects.update(type='cat')

## Задание №4
Добавьте новых питомцев из списка pets_new. Дан список pets_new который содержит словарь с текстовыми полями name и type. Длина списка pets_new - ровно 100 элементов. В этом задании вы ограничены в количестве запросов к базе данных. Задание будет выполнено верно, если количестов запрсов к базе данных не превысит 10.

In [ ]:
# Pet хранит в себе модель питомцев
# pets_new - список из 100 элементов,
# который необходимо занести в базу данных

Pet.objects.bulk_create([
    Pet(name=pet['name'], type=pet['type']) for pet in pets_new
])

## Задание №5
Найдите наиболее часто повыторяющийся тип питомцев. Поменяйте у всех питомцев с этим типом значение поля 'type' на 'horse'. В базе гарантировано есть тип питомцев, которых больше, чем остальных.

Пример:

В базе данных есть 2 питомца с типом 'dog', 4 с типом 'cat' и 11 с типом 'hamster'. Нужно для всех питомцев с типом 'hamster' поменять значение поля 'type' на 'horse'.

In [ ]:
from django.db.models import Count

type_for_change = 'horse'

query1 = Pet.objects.annotate(Count('id')).values('type').order_by('-id')[0]
Pet.objects.filter(type=query1['type']).update(type=type_for_change)

## Задание №6
Удалите кошек(поле type=cat) и рыб(поле type=fish) по имени 'Max'. Регистр имени необходимо НЕ учитывать.

In [ ]:
# Pet хранит в себе модель питомцев
from django.db.models import Q

Pet.objects.filter(
    Q(type='cat') & Q(name__iexact='max') | \
    Q(type='fish') & Q(name__iexact='max')
).delete()